In [1]:
# Loaded all the dependencies:
import os

# Found the latest version of Spark:
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Installed Spark and Java:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Started a SparkSession:
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [2]:
# Imported the packages:
from pyspark.sql import SparkSession
import time

# Created a SparkSession:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a Spark DataFrame:
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), header=True, inferSchema=True, quote='\'', escape='\'')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Created a temporary table/view of the DataFrame in order to run several queries:
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. Queried the average price for a 4-bedroom house sold in each year and rounded the average price to 2 decimal places; also sorted by
# year ascendingly for easy reading:
query = ("""
        SELECT ROUND(AVG(price), 2) AS Average_Price, YEAR(date) AS Year
        FROM home_sales
        WHERE bedrooms==4
        GROUP BY YEAR(date)
        ORDER BY YEAR(date)
        """)
spark.sql(query).show()

+-------------+----+
|Average_Price|Year|
+-------------+----+
|     300263.7|2019|
|    298353.78|2020|
|    301819.44|2021|
|    296363.88|2022|
+-------------+----+



In [6]:
# 4. Queried the average price of a home and the year it was built, but only included those that have 3 bedrooms and 3 bathrooms, and
# rounded the average price to 2 decimal places; also sorted by year ascendingly for easy reading:
query = ("""
        SELECT ROUND(AVG(price), 2) AS Average_Price, date_built AS Date_Built
        FROM home_sales
        WHERE bedrooms==3 AND bathrooms==3
        GROUP BY date_built
        ORDER BY date_built
        """)
spark.sql(query).show()

+-------------+----------+
|Average_Price|Date_Built|
+-------------+----------+
|    292859.62|      2010|
|    291117.47|      2011|
|    293683.19|      2012|
|    295962.27|      2013|
|    290852.27|      2014|
|     288770.3|      2015|
|    290555.07|      2016|
|    292676.79|      2017|
+-------------+----------+



In [7]:
# 5. Queried the average price of a home and the year it was built, but only included those that have 3 bedrooms, 3 bathrooms, 2 floors,
# and are greater than or equal to 2,000 square feet; rounded the average price to 2 decimal places; also sorted by year ascendingly for
# easy reading:
query = ("""
        SELECT ROUND(AVG(price), 2) AS Average_Price, date_built AS Date_Built
        FROM home_sales
        WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living>=2000
        GROUP BY date_built
        ORDER BY date_built
        """)
spark.sql(query).show()

+-------------+----------+
|Average_Price|Date_Built|
+-------------+----------+
|    285010.22|      2010|
|    276553.81|      2011|
|    307539.97|      2012|
|    303676.79|      2013|
|    298264.72|      2014|
|    297609.97|      2015|
|     293965.1|      2016|
|    280317.58|      2017|
+-------------+----------+



In [8]:
# 6. Queried the 'view' rating to get the average price of a home, rounded it to 2 decimal places, and only included the homes, which
# are greater than or equal to $350,000; also determined the run time for this query:
start_time = time.time()
query = ("""SELECT ROUND(AVG(price), 2) AS Average_Price, view as View
            FROM home_sales
            WHERE price>=350000
            GROUP BY view
         """)
spark.sql(query).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|View|
+-------------+----+
|    399856.95|  31|
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|    401419.75|  34|
|   1053472.79|  81|
|    402124.62|  28|
|   1058802.78|  76|
|    401506.97|  26|
|    399537.66|  27|
|    400598.05|  44|
|    401501.32|  12|
|   1137372.73|  91|
|    402022.68|  22|
|   1026006.06|  93|
|     398447.5|  47|
|    401044.25|   1|
|    733780.26|  52|
|    398917.98|  13|
+-------------+----+
only showing top 20 rows

--- 0.5615453720092773 seconds ---


In [9]:
# 7. Cached the temporary table home_sales:
spark.sql('cache table home_sales')

DataFrame[]

In [10]:
# 8. Checked if the table was cached:
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. By using the cached data, ran the query that filtered out the 'view' ratings with the average price greater than or equal to
# $350,000; determined the runtime, and compared it to the uncached runtime:
start_time = time.time()
query = ("""SELECT ROUND(AVG(price), 2) AS Average_Price, view as View
            FROM home_sales
            WHERE price>=350000
            GROUP BY view
         """)
spark.sql(query).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|View|
+-------------+----+
|    399856.95|  31|
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|    401419.75|  34|
|   1053472.79|  81|
|    402124.62|  28|
|   1058802.78|  76|
|    401506.97|  26|
|    399537.66|  27|
|    400598.05|  44|
|    401501.32|  12|
|   1137372.73|  91|
|    402022.68|  22|
|   1026006.06|  93|
|     398447.5|  47|
|    401044.25|   1|
|    733780.26|  52|
|    398917.98|  13|
+-------------+----+
only showing top 20 rows

--- 0.3539855480194092 seconds ---


In [12]:
# 10. Partitioned by the 'date_built' field on the formatted parquet 'home sales' data:
df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

In [13]:
# 11. Read the parquet formatted data:
df_p = spark.read.parquet('home_sales_partitioned')

In [14]:
# 12. Created a temporary table/view for the parquet data:
df_p.createOrReplaceTempView('home_sales_p')

In [15]:
# 13. Ran the query that filtered out the 'view' ratings with the average price of greater than or equal to $350,000 with the parquet
# DataFrame; determined the runtime, and compared it to the cached version:
start_time = time.time()
query = ("""SELECT ROUND(AVG(price), 2) AS Average_Price, view as View
            FROM home_sales_p
            WHERE price>=350000
            GROUP BY view
         """)
spark.sql(query).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|View|
+-------------+----+
|    399856.95|  31|
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|    401419.75|  34|
|   1053472.79|  81|
|    402124.62|  28|
|   1058802.78|  76|
|    401506.97|  26|
|    399537.66|  27|
|    400598.05|  44|
|    401501.32|  12|
|   1137372.73|  91|
|    402022.68|  22|
|   1026006.06|  93|
|     398447.5|  47|
|    401044.25|   1|
|    733780.26|  52|
|    398917.98|  13|
+-------------+----+
only showing top 20 rows

--- 0.9575011730194092 seconds ---


In [16]:
# 14. Uncached the 'home_sales' temporary table:
spark.sql('uncache table home_sales')

DataFrame[]

In [17]:
# 15. Checked if the 'home_sales' was no longer cached:
spark.catalog.isCached('home_sales')

False